In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from collections import defaultdict

## Attribution

Some code in this notebook adapted from DSCI 525 Lectures [1](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html) and [2](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture2.html) notes

<br>
<br>
<br>
<br>
<br>

## 3. Downloading the data

In [3]:
%cd /media/dan/data/School/Block_6/DSCI525/DSCI_525_Group_007
## Change it to the location that you want to download your files to.

[Errno 2] No such file or directory: '/media/dan/data/School/Block_6/DSCI525/DSCI_525_Group_007'
/Users/wanggaoxiang/DSCI_term6/525/DSCI_525_Group_007/notebooks


In [4]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [5]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [6]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 2.59 s, sys: 4.34 s, total: 6.93 s
Wall time: 35.6 s


In [7]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 13.5 s, sys: 3.1 s, total: 16.6 s
Wall time: 17.1 s


In [8]:
%ls -ltr figshareairline/

total 12125320
-rw-r--r--   1 wanggaoxiang  staff  814041183 27 Mar 21:13 data.zip
-rw-r--r--   1 wanggaoxiang  staff   95376895 27 Mar 21:13 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff   94960113 27 Mar 21:13 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff   82474546 27 Mar 21:13 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff  127613760 27 Mar 21:13 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff  232118894 27 Mar 21:13 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff  330360682 27 Mar 21:14 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff  254009247 27 Mar 21:14 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff  235661418 27 Mar 21:14 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff  294260911 27 Mar 21:14 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 wanggaoxiang  staff  295768615 27 Mar 21:14 EC-Earth3-

<br>
<br>
<br>
<br>
<br>

## 4. Loading the data with Pandas

In [9]:
%%time
## This is all the data, as is

files = glob.glob('figshareairline/*.csv')
files.remove("figshareairline/observed_daily_rainfall_SYD.csv")
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv")

CPU times: user 6min 2s, sys: 25.2 s, total: 6min 27s
Wall time: 6min 34s


| Team Member |	Operating System | RAM | Processor | Is SSD  | Time taken 
|---|---|---|---|---|---|
| Alex  |  MacOS (13.1) | 24 GB  | M2  | Yes  | 3min 14s  |
| Daniel  |Ubuntu 22.04.2 LTS|20 GB|Intel® Core™ i5-6200U CPU @ 2.30GHz × 4|Yes|10min 41s|
| Ranjit  |Windows 11 Education       | 16 GB  |Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz   2.59 GHz   |Yes   |5 mins 50 s   |
| Gaoxiang |  MacOS  13.1 (22C65)  | 32 GB 2400 MHz DDR4|  Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz |  Yes |  6min 34s |



### Summary

Processor speed appeared to matter more in this instance than the amount of RAM or the OS. 

The fastest laptop was the M2 MacBook by over two minutes, then the two Intel i7 chips were within 40 seconds (the faster of these two did, in fact, perform faster).  The slowest was the older i5 Intel chip.  

Between the two intel i7 computers, Ranjit's had half the amount of RAM as Gaoxiang but performed more quickly, indicating that RAM was not a limiting factor for a dataset of this size. 

<br>
<br>
<br>
<br>
<br>

## 5. Python EDA

In [10]:
%%time

df.describe()

CPU times: user 10.1 s, sys: 1.82 s, total: 11.9 s
Wall time: 11.9 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [11]:
%%time

df[['lat_min','lat_max','lon_min','lon_max','rain (mm/day)']].astype('float32', errors='ignore').describe()

CPU times: user 8.78 s, sys: 1.91 s, total: 10.7 s
Wall time: 10.7 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310497e+01,-3.197765e+01,1.469057e+02,1.482150e+02,1.901173e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [12]:
%%time

df.loc[:,'rain (mm/day)'].describe()

CPU times: user 2.52 s, sys: 209 ms, total: 2.73 s
Wall time: 2.73 s


count    5.924854e+07
mean     1.901170e+00
std      5.585735e+00
min     -3.807373e-12
25%      3.838413e-06
50%      6.154947e-02
75%      1.020918e+00
max      4.329395e+02
Name: rain (mm/day), dtype: float64

| Team Member |	Operating System | RAM | Processor | Is SSD  | Time taken (total; casted; single column)
|---|---|---|---|---|---|
| Alex  |  MacOS (13.1) | 24 GB  | M2  | Yes  | 8.62 s; 7.81 s; 2.2 s  |
| Daniel  |Ubuntu 22.04.2 LTS|20 GB|Intel® Core™ i5-6200U CPU @ 2.30GHz × 4|Yes|24.8 s; 13.4 s; 3.2 s|
| Ranjit  | Windows 11 Education        | 16 GB   | Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz   2.59 GHz      | Yes   |  12.3 s; 9.18 s; 2.73 s|
| Gaoxiang |  MacOS 13.1 (22C65)   |  32 GB 2400 MHz DDR4 |  Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz |  Yes | 11.9 s; 10.7 s; 2.73 s  |



### Summary:

The fastest approach that we tested was to use only the column that we wanted to summarize (which in this case was `rain (mm/day)`). This made a large difference on all laptops tested, to the point where absolute speed difference between them was negligible. 

Casting the data as float 32s improved speed, but this improvement was more noticeable on Daniel's laptop (which has an older processor). On Ranjit's, Alex's, and Gaoxiang's laptops this did not make a large difference. 

<br>
<br>
<br>
<br>
<br>

## 6. R EDA

### Why we chose Arrow

We will run analysis using both python (with pandas) and R. Using both languages with CSVs creates speed-bumps whereby we need to serialize and deserialize the data. One useful property of Arrow is that it's language-independent and does not require re-serialization, which speeds up the analysis as this is a large (and potentially slow) dataset to work with. 

In [17]:
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [19]:
filepathcsv = "figshareairline/combined_data.csv"

In [28]:
import pyarrow.dataset as ds
import pyarrow as pa
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [29]:
%%time

dataset = ds.dataset(filepathcsv, format="csv")
# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()
# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 18.6 s, sys: 2.83 s, total: 21.4 s
Wall time: 20 s


In [30]:
%%time

%%R -i r_table

suppressMessages(library(dplyr))
result <- r_table %>% count(model)
print(result %>% collect())


# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 3.3 s, sys: 108 ms, total: 3.41 s
Wall time: 749 ms


In [31]:
%%time

%%R -i r_table

suppressMessages(library(dplyr))
result <- r_table %>% count(model)
print(result %>% collect())


# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 3.02 s, sys: 29.2 ms, total: 3.05 s
Wall time: 357 ms


In [32]:
%%R -i r_table

result

Table (query)
model: string
n: int32

See $.data for the source Arrow object
